Break ties and resolve ambiguity by adding weighted set cover functionality to protein grouping

In [ ]:
import numpy as np
import pandas as pd
from typing import List, Any
from numpy.typing import NDArray
from alphadia.grouping import perform_grouping

In [ ]:
def group_and_parsimony_new(
    precursor_idx: NDArray[np.int64],
    precursor_ids: NDArray[Any],
    precursor_scores: NDArray[np.float64] = None,
):
    """
    Function to group ids based on precursor indices and return groups & master ids as lists

    Parameters
    ----------

        precursor_idx : np.array[int]
            array containing unique integer indices corresponding to each peptide precursor

        precursor_ids : np.array[str]
            array of variable length semicolon separated str belonging to a given peptide precursor id

        precursor_scores : np.array[float]
            array of precursor scores corresponding to each precursor_idx; optinoal argument, defaults to 1.0 if not provided

    Returns
    -------

        ids : list[str]
            list of ids linked to a given peptide precursor, such that each precursor only belongs to one id. This list is ordered by precursor_idx.

        groups : list[str]
            list of semicolon separated ids belonging to a given peptide precursor, such that each precursor only belongs to one group. This list is ordered by precursor_idx.

    """

    # reshape precursor indices and ids into a dictionary of ids linked to sets of precursors
    id_dict = {}
    for precursor, ids, scores in zip(precursor_idx, precursor_ids, precursor_scores):
        for id in ids.split(";"):
            if id not in id_dict:
                id_dict[id] = ([], [])
            id_dict[id][0].append(precursor)
            id_dict[id][1].append(scores)
    print("id dict done")

    # perform greedy set cover on protein_dict: if two ids share a precursor, the precorsor is deleted from the id with fewer precursors. If this removes all precursors from an id, it is added to the larger id's group.
    id_group = []
    id_master = []
    precursor_indices = []
    precursor_score = []

    # loop bounds max iterations
    for _ in range(len(id_dict)):
        # remove longest set from dict as query & remove query peptided from all other sets
        # query_id = max(id_dict.keys(), key = lambda x : len(id_dict[x][0]))
        query_id = max(id_dict.keys(), key=lambda x: np.sum(id_dict[x][1]))
        query_group = [query_id]

        query_val = id_dict.pop(query_id)
        query_peptides = query_val[0]
        query_scores = query_val[1]

        if len(query_peptides) == 0:
            break

        # update peptides & add to group if depleted
        for subject_protein, subject_values in id_dict.items():
            subject_peptides = subject_values[0]
            subject_scores = subject_values[1]

            if not subject_peptides:
                continue
            keep_mask = np.isin(subject_peptides, query_peptides, invert=True)
            new_subject_peptides = [
                item for item, keep in zip(subject_peptides, keep_mask) if keep
            ]
            new_subject_scores = [
                item for item, keep in zip(subject_scores, keep_mask) if keep
            ]
            new_values = (new_subject_peptides, new_subject_scores)
            id_dict[subject_protein] = new_values
            if not new_subject_peptides:
                query_group.append(subject_protein)

        # save query to output lists
        id_group.append(query_group)
        id_master.append(query_id)
        precursor_indices.append(query_peptides)
        precursor_score.append(query_scores)

    # convert id_group sublists to semicolon separated strings
    id_group = [";".join(x) for x in id_group]

    # reshape output data and align with precursor dataframe input. Use dictionary for efficient ordering
    return_dict = {}
    for i, values in enumerate(zip(precursor_indices, precursor_score)):
        for precursor, score in zip(*values):
            return_dict[precursor] = (id_master[i], id_group[i], score)

    # check that all precursors are found again
    if len(return_dict) != len(precursor_idx):
        raise ValueError(
            "Not all precursors were found in the output of the grouping function."
        )

    # order by precursor index
    return_dict_ordered = {key: return_dict[key] for key in precursor_idx}
    ids, groups, scores = zip(*return_dict_ordered.values())

    return ids, groups


def perform_grouping_new(
    psm: pd.DataFrame,
    genes_or_proteins: str = "proteins",
    score_column: Any = "proba",
):
    """Highest level function for grouping proteins in precursor table

    Parameters:
        gene_or_protein (str, optional): Column to group proteins by. Defaults to "proteins".

    """

    if genes_or_proteins not in ["genes", "proteins"]:
        raise ValueError("Selected column must be 'genes' or 'proteins'")

    # if no score column is provided, all precursors are weighted equally
    if score_column is None:
        psm[score_column] = 1.0

    # create non-duplicated view of precursor table: precursor indices are unique
    duplicate_mask = ~psm.duplicated(subset=["precursor_idx"], keep="first")
    upsm = psm.loc[
        duplicate_mask, ["precursor_idx", genes_or_proteins, "_decoy", score_column]
    ]

    # handle case with only one decoy class:
    unique_decoys = upsm["_decoy"].unique()
    if len(unique_decoys) == 1:
        upsm["_decoy"] = -1
        upsm["pg_master"], upsm["pg"] = group_and_parsimony_new(
            upsm.precursor_idx.values,
            upsm[genes_or_proteins].values,
            upsm[score_column].values,
        )
        upsm = upsm[["precursor_idx", "pg_master", "pg"]]
    else:
        target_mask = upsm["_decoy"] == 0
        decoy_mask = upsm["_decoy"] == 1

        t_df = upsm[target_mask].copy()
        new_columns = group_and_parsimony_new(
            t_df.precursor_idx.values,
            t_df[genes_or_proteins].values,
            upsm[score_column].values,
        )
        t_df["pg_master"], t_df["pg"] = new_columns

        d_df = upsm[decoy_mask].copy()
        new_columns = group_and_parsimony_new(
            d_df.precursor_idx.values,
            d_df[genes_or_proteins].values,
            upsm[score_column].values,
        )
        d_df["pg_master"], d_df["pg"] = new_columns

        upsm = pd.concat([t_df, d_df])[["precursor_idx", "pg_master", "pg"]]

    return psm.merge(upsm, on="precursor_idx", how="left")

For weighted grouping, rather than going by set length in ranking, there has to be a column summarizing metrics contributed by each peptide, and the set with the highest metric is chosen first. Peptides still have to be removed as is, but the corresponding instance of the metric has to be removed before the next selection step as well. 

This necessitates rewriting the grouping function to get away from set comparisons, since a numeric metric needs to be removed index-based.

Simulate simple case where the input precursor dataframe also contains a numeric metric column.

simulate subsumable case where the first protein is chosen first, and the metric establishes an empiric ordering

In [ ]:
# | protein | peptides |
# | ------- | -------- |
# | P1      | 1,2      |
# | P2      |   2,3    |
# | P3      |     3,4  |

Depending on where the counting starts, this can resolve into a P1 / P3 outcome, or - if one starts with P2 - a P2 / P1 / P3 outcome.
From a parsimony perspective, the latter is incorrect as we try to explain the set of peptides with the smallest number of proteins.
However, by adding a score metric, we can resolve this issue and reformulate parsimony into "let us take the fewest and best proteins to explain the data"

In [ ]:
# | precursor_idx | proteins | score |
# | ------------- | -------- | ----- |
# | 1             | P1       | 0.4   |
# | 2             | P1;P2    | 0.7   |
# | 3             | P2;P3    | 0.5   |
# | 4             | P3       | 0.2   |

precursor_idx = [1, 2, 3, 4]
proteins = ["P1", "P1;P2", "P2;P3", "P3"]
decoy = [0, 0, 0, 0]
proba = [0.4, 0.7, 0.5, 0.2]
df = pd.DataFrame(
    {
        "precursor_idx": precursor_idx,
        "proteins": proteins,
        "proba": proba,
        "_decoy": decoy,
    }
)

# apply standard grouping with ordered and reordered dataframe to illustrate the issue
print("standard grouping: 1 and 3 get all peptides, subsumaple protein 2 is removed")
display(perform_grouping(df, genes_or_proteins="proteins"))

print("reordered grouping: start with protein 2 and all proteins are in the output")
df_reordered = df.iloc[[2, 3, 0, 1]]
display(
    perform_grouping(df_reordered, genes_or_proteins="proteins").sort_values(
        by="precursor_idx"
    )
)

standard grouping: 1 and 3 get all peptides, subsumaple protein 2 is removed


,precursor_idx,proteins,proba,_decoy,pg_master,pg
0,1,P1,0.4,0,P1,P1
1,2,P1;P2,0.7,0,P1,P1
2,3,P2;P3,0.5,0,P3,P3;P2
3,4,P3,0.2,0,P3,P3;P2


reordered grouping: start with protein 2 and all proteins are in the output


,precursor_idx,proteins,proba,_decoy,pg_master,pg
2,1,P1,0.4,0,P1,P1
3,2,P1;P2,0.7,0,P2,P2
0,3,P2;P3,0.5,0,P2,P2
1,4,P3,0.2,0,P3,P3


In [ ]:
# in the new version of grouping, the output is constant:

# apply standard grouping with ordered and reordered dataframe to illustrate the issue
print("standard grouping: 1 and 3 get all peptides, subsumaple protein 2 is removed")
display(perform_grouping_new(df, genes_or_proteins="proteins", score_column="proba"))

print("reordered grouping: start with protein 2 and all proteins are in the output")
df_reordered = df.iloc[[2, 3, 0, 1]]
display(
    perform_grouping_new(
        df_reordered, genes_or_proteins="proteins", score_column="proba"
    ).sort_values(by="precursor_idx")
)

standard grouping: 1 and 3 get all peptides, subsumaple protein 2 is removed


,precursor_idx,proteins,proba,_decoy,None,pg_master,pg
0,1,P1,0.4,0,1.0,P1,P1
1,2,P1;P2,0.7,0,1.0,P2,P2
2,3,P2;P3,0.5,0,1.0,P2,P2
3,4,P3,0.2,0,1.0,P3,P3


reordered grouping: start with protein 2 and all proteins are in the output


,precursor_idx,proteins,proba,_decoy,None,pg_master,pg
2,1,P1,0.4,0,1.0,P1,P1
3,2,P1;P2,0.7,0,1.0,P2,P2
0,3,P2;P3,0.5,0,1.0,P2,P2
1,4,P3,0.2,0,1.0,P3,P3


In [ ]:
# test timing with real dataset
psm = pd.read_csv(
    "/Users/vincenthbrennsteiner/Documents/mann_labs/data/alphadia_example_output/psm.tsv",
    sep="\t",
)
perform_grouping_new(psm.iloc[:2000], genes_or_proteins="proteins")

id dict done


,base_width_mobility,base_width_rt,rt_observed,mobility_observed,mono_ms1_intensity,top_ms1_intensity,sum_ms1_intensity,weighted_ms1_intensity,weighted_mass_deviation,weighted_mass_error,...,flat_frag_start_idx,channel,i_2,mz_library,_decoy,proba,qval,run,pg_master,pg
0,0.0,61.264650,4150.9365,0.000001,1.686096e+08,1.686096e+08,4.144033e+08,1.288110e+08,-0.059417,0.059417,...,355530,0,0.194793,946.43380,0.0,0.000028,0.0,HF_20210523_M300-Y700_DIA_3,P16521,P16521
1,0.0,39.634766,3776.8835,0.000001,2.705160e+07,2.659347e+07,7.035453e+07,2.097147e+07,-0.171322,0.171322,...,498472,0,0.210362,1094.04240,0.0,0.000030,0.0,HF_20210523_M300-Y700_DIA_3,P38011,P38011
2,0.0,36.285156,4728.6430,0.000001,3.778682e+07,3.778682e+07,9.161826e+07,2.898220e+07,-0.038115,0.038115,...,470025,0,0.188118,900.42770,0.0,0.000031,0.0,HF_20210523_M300-Y700_DIA_3,P33331,P33331
3,0.0,61.782715,5049.4670,0.000001,1.528510e+08,1.718843e+08,4.294707e+08,1.260246e+08,-0.080193,0.080193,...,339007,0,0.213075,1018.02515,0.0,0.000033,0.0,HF_20210523_M300-Y700_DIA_3,P14540,P14540
4,0.0,40.335450,3715.7993,0.000001,5.214854e+07,5.214854e+07,1.355514e+08,4.043824e+07,-0.192274,0.192274,...,278590,0,0.200084,931.41960,0.0,0.000049,0.0,HF_20210523_M300-Y700_DIA_3,P05759,P05759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.0,15.397461,6522.5070,0.000001,2.942341e+05,2.942341e+05,3.853428e+05,1.930822e+05,-0.314520,0.314520,...,1453123,0,0.099229,498.33423,0.0,0.000260,0.0,HF_20210523_M300-Y700_DIA_3,Q9CQ79,Q9CQ79
1996,0.0,35.963380,2560.8516,0.000001,3.377257e+07,3.377257e+07,5.121644e+07,2.452357e+07,0.108860,0.108860,...,1466471,0,0.091525,493.78510,0.0,0.000260,0.0,HF_20210523_M300-Y700_DIA_3,Q9CZX7,Q9CZX7
1997,0.0,23.774414,6316.7260,0.000001,2.752156e+07,2.752156e+07,5.553502e+07,2.057251e+07,-1.254437,1.254437,...,583487,0,0.144702,689.38480,0.0,0.000260,0.0,HF_20210523_M300-Y700_DIA_3,P46096,P46096;P46097
1998,0.0,23.593872,1264.7338,0.000001,2.403789e+07,2.403789e+07,4.926449e+07,1.795892e+07,0.442849,0.442849,...,172513,0,0.153876,753.89920,0.0,0.000260,0.0,HF_20210523_M300-Y700_DIA_3,O35526,O35526
